In [1]:
import geopandas as gpd
import os
import zipfile as zf
import geopandas
from shapely.geometry import shape
import gzip 
import urllib.request
import xarray as xr
import io
import shapefile
import pandas as pd
import json

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [35]:
df = pd.read_pickle("final.pkl")

In [11]:
df

,time,timeMin,timeMax,gdp
0,1980-01-15,-1.59,1.77,6837.641000
1,1980-02-15,-2.59,6.66,6790.678333
2,1980-03-15,-1.40,3.26,6743.715667
3,1980-04-15,-1.39,3.05,6696.753000
4,1980-05-15,-1.49,2.39,6694.100000
...,...,...,...,...
491,2020-12-15,-0.27,5.58,18989.854000
492,2021-01-15,0.48,7.10,19087.568000
493,2021-02-15,-4.14,1.27,NaN
494,2021-03-15,-1.28,5.49,NaN


In [36]:
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df = df.set_index(df['time'])
df = df.sort_index()
df =  df.drop(["time","index","gdp"], axis =1)

In [37]:
df

,timeMin,timeMax
time,,
1980-01-15,-1.59,1.77
1980-02-15,-2.59,6.66
1980-03-15,-1.40,3.26
1980-04-15,-1.39,3.05
1980-05-15,-1.49,2.39
...,...,...
2020-12-15,-0.27,5.58
2021-01-15,0.48,7.10
2021-02-15,-4.14,1.27


In [25]:
496*0.75

372.0

Scaling dataset

In [38]:
min_max_scaler = MinMaxScaler()
df[["timeMin", "timeMax"]] = min_max_scaler.fit_transform(df[["timeMin", "timeMax"]])

In [39]:
df

,timeMin,timeMax
time,,
1980-01-15,0.662583,0.124825
1980-02-15,0.568332,0.467742
1980-03-15,0.680490,0.229313
1980-04-15,0.681433,0.214586
1980-05-15,0.672008,0.168303
...,...,...
2020-12-15,0.786993,0.392006
2021-01-15,0.857681,0.498597
2021-02-15,0.422243,0.089762


In [40]:
train,test = df[1:372], df[372:] 

In [41]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [42]:
n_past = 50
n_future = 1
n_features = 2

In [43]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [44]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 41200       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 1, 100)       0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1, 100)       80400       repeat_vector[0][0]              
                                                                 lstm[0][1]            

In [45]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 50, 100), (N 41200       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 100), (None, 80400       lstm_2[0][0]                     
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 1, 100)       0           lstm_3[0][0]                     
_______________________________________________________________________________________

In [66]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])

Epoch 1/25
11/11 [==============================] - 1s 126ms/step - loss: 37.9084 - val_loss: 38.2174
Epoch 2/25
11/11 [==============================] - 0s 28ms/step - loss: 35.0399 - val_loss: 34.2152
Epoch 3/25
11/11 [==============================] - 0s 30ms/step - loss: 31.6370 - val_loss: 32.7484
Epoch 4/25
11/11 [==============================] - 0s 29ms/step - loss: 30.1233 - val_loss: 31.9056
Epoch 5/25
11/11 [==============================] - 0s 27ms/step - loss: 29.3625 - val_loss: 31.1961
Epoch 6/25
11/11 [==============================] - 0s 28ms/step - loss: 28.8182 - val_loss: 30.7043
Epoch 7/25
11/11 [==============================] - 0s 28ms/step - loss: 28.3626 - val_loss: 30.3507
Epoch 8/25
11/11 [==============================] - 0s 30ms/step - loss: 27.9872 - val_loss: 30.0786
Epoch 9/25
11/11 [==============================] - 0s 34ms/step - loss: 27.6826 - val_loss: 29.8053
Epoch 10/25
11/11 [==============================] - 0s 33ms/step - loss: 27.4255 - val_lo

In [59]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

In [65]:
pred_e1d1

array([[[-0.78482825,  8.925818  ]],

       [[-0.7875119 ,  8.873264  ]],

       [[-0.69857603,  8.803901  ]],

       [[-0.6549057 ,  8.798472  ]],

       [[-0.6062141 ,  8.785261  ]],

       [[-0.62385094,  8.757107  ]],

       [[-0.6580555 ,  8.740157  ]],

       [[-0.66045976,  8.780213  ]],

       [[-0.6212725 ,  8.822549  ]],

       [[-0.63400286,  8.84796   ]],

       [[-0.52393407,  8.937671  ]],

       [[-0.3589427 ,  9.041061  ]],

       [[-0.20861697,  9.054074  ]],

       [[-0.06694087,  9.023523  ]],

       [[-0.10685147,  9.027793  ]],

       [[-0.17763846,  8.984735  ]],

       [[-0.18189195,  8.955389  ]],

       [[-0.20622902,  8.965231  ]],

       [[-0.26444742,  8.9573555 ]],

       [[-0.3201954 ,  8.950083  ]],

       [[-0.3170923 ,  8.992918  ]],

       [[-0.22994462,  9.03771   ]],

       [[-0.38339764,  9.044752  ]],

       [[-0.4300729 ,  9.026008  ]],

       [[-0.37601852,  9.037577  ]],

       [[-0.53886026,  9.010204  ]],

       [[-0.

In [62]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    #pred1_e1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    #pred1_e2d2[:,:,index]=scaler.inverse_transform(pred1_e2d2[:,:,index])
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [64]:
pred_e1d1

array([[[-0.78482825,  8.925818  ]],

       [[-0.7875119 ,  8.873264  ]],

       [[-0.69857603,  8.803901  ]],

       [[-0.6549057 ,  8.798472  ]],

       [[-0.6062141 ,  8.785261  ]],

       [[-0.62385094,  8.757107  ]],

       [[-0.6580555 ,  8.740157  ]],

       [[-0.66045976,  8.780213  ]],

       [[-0.6212725 ,  8.822549  ]],

       [[-0.63400286,  8.84796   ]],

       [[-0.52393407,  8.937671  ]],

       [[-0.3589427 ,  9.041061  ]],

       [[-0.20861697,  9.054074  ]],

       [[-0.06694087,  9.023523  ]],

       [[-0.10685147,  9.027793  ]],

       [[-0.17763846,  8.984735  ]],

       [[-0.18189195,  8.955389  ]],

       [[-0.20622902,  8.965231  ]],

       [[-0.26444742,  8.9573555 ]],

       [[-0.3201954 ,  8.950083  ]],

       [[-0.3170923 ,  8.992918  ]],

       [[-0.22994462,  9.03771   ]],

       [[-0.38339764,  9.044752  ]],

       [[-0.4300729 ,  9.026008  ]],

       [[-0.37601852,  9.037577  ]],

       [[-0.53886026,  9.010204  ]],

       [[-0.